In [1]:
%pip install crewai crewai_tools firecrawl-py
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import sys, importlib, importlib.metadata as m
# print("python:", sys.executable)
# print("version():", m.version("crewai"))  # metadata view
# import crewai
# print("module file:", crewai.__file__)    # which package file is imported


In [3]:
# import sys, subprocess
# subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "crewai"])
# # run uninstall twice if it still finds another copy
# subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "crewai"])

# subprocess.check_call([sys.executable, "-m", "pip", "install",
#                        "--no-cache-dir", "--force-reinstall", "crewai==0.177.0"])


In [4]:
# import sys, importlib, importlib.metadata as m
# import crewai_tools
# print("py:", sys.executable)

# # Where is crewai_tools coming from?
# print("crewai_tools file:", crewai_tools.__file__)

# # Package metadata version (note: PyPI name uses a dash)
# print("crewai-tools version:", m.version("crewai-tools"))

# # What symbols are exported?
# print("has DirectoryReadTool?", "DirectoryReadTool" in dir(crewai_tools))


In [5]:
# import sys, subprocess
# subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "crewai-tools"])
# # run twice in case multiple copies exist
# subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "crewai-tools"])

# # reinstall the latest tools + extras
# subprocess.check_call([sys.executable, "-m", "pip", "install",
#                        "--no-cache-dir", "--force-reinstall",
#                        "crewai", "crewai-tools"])

In [6]:
# import yaml
# from pathlib import Path

# p = Path("config/planner_tasks.yaml")
# print("Absolute path:", p.resolve())
# print("Exists:", p.exists(), "Size:", p.stat().st_size if p.exists() else "N/A")

# raw = p.read_text()
# print("First 200 chars:\n", raw[:200])

# # Confirm you're using PyYAML (not the unrelated 'yaml' package)
# print("yaml module file:", getattr(yaml, "__file__", None))
# print("yaml __version__:", getattr(yaml, "__version__", "unknown"))
# try:
#     print("PyPI package 'PyYAML' version:", m.version("PyYAML"))
# except m.PackageNotFoundError:
#     print("PyYAML not found via metadata")

# # Load ALL YAML documents (handles '---' multi-doc files)
# docs = [d for d in yaml.safe_load_all(raw) if d is not None]
# print("Documents loaded:", docs)


In [7]:
# import sys, subprocess
# subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "crewai-tools"])
# # run twice in case multiple copies exist
# subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "crewai-tools"])

# # reinstall the latest tools + extras
# subprocess.check_call([sys.executable, "-m", "pip", "install",
#                        "--no-cache-dir", "--force-reinstall",
#                        "crewai", "crewai-tools"])

In [8]:
# import sys, subprocess
# subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "crewai-tools"])
# # run twice in case multiple copies exist
# subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "crewai-tools"])

# # reinstall the latest tools + extras
# subprocess.check_call([sys.executable, "-m", "pip", "install",
#                        "--no-cache-dir", "--force-reinstall",
#                        "crewai", "crewai-tools"])

In [9]:
from crewai import LLM  # should now work
llm = LLM(model="gpt-4o")

In [10]:
import yaml

with open('config/planner_agents.yaml', 'r') as f:
    agents_config = yaml.safe_load(f)
    print(agents_config)

with open('config/planner_tasks.yaml', 'r') as f:
    tasks_config = yaml.safe_load(f)
    print(tasks_config)

{'draft_analyzer': {'role': 'Draft Analyzer\n', 'goal': 'Analyze draft and identify key ideas, sections and technical concepts\n', 'backstory': 'You are a technical writer with years of experience writing, editing and reviewing technical blogs. You have a talent for understanding and documenting technical concepts.\n', 'verbose': True}, 'twitter_thread_planner': {'role': 'Twitter Thread Planner\n', 'goal': 'Create a Twitter thread plan based on the provided draft analysis\n', 'backstory': 'You are a technical writer with years of experience in converting long technical blogs into Twitter threads. You have a talent for breaking longform content into bite-sized tweets that are engaging and informative. And identify relevant urls to media that can be associated with a tweet.\n', 'verbose': True}, 'linkedin_post_planner': {'role': 'LinkedIn Post Planner\n', 'goal': 'Create an engaging LinkedIn post based on the provided draft analysis\n', 'backstory': 'You are a technical writer with exten

In [11]:
from pydantic import BaseModel
from typing import Optional

class Tweet(BaseModel):
    """Represents an individual tweet in a thread"""
    content: str
    is_hook: bool = False 
    media_urls: Optional[list[str]] = []

class Thread(BaseModel):
    """Represents a Twitter thread"""
    topic: str 
    tweets: list[Tweet]

class LinkedInPost(BaseModel):
    """Represents a LinkedIn post"""
    content: str  # The main content of the post
    media_url: str # Main image URL for the post



In [12]:
blog_post_url = "https://blog.dailydoseofds.com/p/5-chunking-strategies-for-rag"

In [13]:
import crewai_tools
# print(f"dir = {dir(crewai_tools)}\n")
from crewai_tools import DirectoryReadTool, FileReadTool 

all_tools = [DirectoryReadTool(), FileReadTool()]

Purpose: Extract key takeaways and media references from the blog post.

Output: No structured output (acts as preprocessing).

Usage: Feeds extracted data into both the Twitter and LinkedIn planners.

In [14]:
from crewai import Agent, Task
draft_analyzer = Agent(config=agents_config['draft_analyzer'],
                       tools=all_tools,
                       llm=llm)

analyze_draft = Task(config=tasks_config['analyze_draft'],
                     agent=draft_analyzer
                     )

Purpose: Convert the blog post into a structured Twitter thread.

Output Format: Thread (structured list of tweets).

In [15]:
from crewai import Agent, Task

twitter_thread_planner = Agent(config=agents_config['twitter_thread_planner'],
                               tools=all_tools,
                               llm=llm)

create_twitter_thread_plan = Task(config=tasks_config['create_twitter_thread_plan'],
                                  agent=twitter_thread_planner,
                                  output_pydantic=Thread)

Purpose: Generate a LinkedIn post optimized for professionals.

Output Format: LinkedInPost (structured long-form content).

In [16]:
from crewai import Agent, Task

linkedin_post_planner = Agent(config=agents_config['linkedin_post_planner'],
                              tools=all_tools,
                              llm=llm)

create_linkedin_post_plan = Task(config=tasks_config['create_linkedin_post_plan'],
                                 agent=linkedin_post_planner,
                                 output_pydantic=LinkedInPost)


In [17]:
from crewai import Agent, Task, Crew

twitter_planning_crew = Crew(

    agents=[draft_analyzer, twitter_thread_planner],
    tasks=[analyze_draft, create_twitter_thread_plan],
    verbose=True

)

linkedin_planning_crew = Crew (

    agents=[draft_analyzer, linkedin_post_planner],
    tasks=[analyze_draft, create_linkedin_post_plan],
    verbose=True

)

This CrewAI Flow orchestrates the entire workflow by:

Scraping the blog post using FireCrawl.

Selecting the target platform (Twitter or LinkedIn).

Generating structured content using AI agents.

Saving the final content plan for future use.

In CrewAI Flows, the state stores persistent values that are needed across different steps. Instead of using an unstructured dictionary, we define a structured state model using Pydantic as follows:

In [18]:
from pydantic import BaseModel
from pathlib import Path

class ContentPlanningState(BaseModel):
    """
    State for the content planning flow
    """
    # URL of the blog to scrape
    blog_post_url: str = blog_post_url
    
    # Path where the scraped content will be stored
    draft_path: Path = "assets/"
    
    # Determines whether to create a Twitter or LinkedIn post 
    post_type: str = "twitter"  
    
    # Example Twitter threads for style reference
    path_to_example_threads: str = "assets/example_threads.txt" 
    
    # Example LinkedIn posts for reference
    path_to_example_linkedin: str = "assets/example_linkedin.txt"

the first step in our flow is to fetch the blog content using FireCrawl:

FireCrawl scrapes the blog post and converts it into Markdown and HTML.

The title is extracted from metadata (or a fallback ID is used).

The content is saved locally at assets/{title}.md for later processing.

Once the blog is scraped, we need to determine whether to generate a Twitter thread or a LinkedIn post.

To do this, we use the router decorator:

If post_type == "twitter", the flow proceeds to generate a Twitter thread.

If post_type == "linkedin", the flow creates a LinkedIn post.

This ensures that only one of the content generation flows executes based on the user's selection.

Now we shall have two methods that will "listen" to the output of the above router method.

If the user has selected Twitter, the twitter_draft method will execute, which has been decorated with this decorator—@listen("twitter"):

The Twitter Planning Crew accepts the blog draft and generates a structured Thread output.

Each tweet is printed, showing:

The content of the tweet.

The associated media URLs (if any).

If the user has selected LinkedIn, the linkedin_draft method will execute, , which has been decorated with this decorator—@listen("linkedin"):

The LinkedIn Planning Crew processes the blog and generates a structured LinkedInPost output.

The generated LinkedIn post is printed for review.

Once the content is generated, the plan is saved as a JSON file for later use.

To do this, we make use of the listen decorator along with a conditional OR logic, which means if either linkedin method or twitter method finishes execution, the save method should run

In [27]:
from crewai.flow.flow import Flow, listen, start, router, or_
from firecrawl import FirecrawlApp
import os
import uuid
import json

class CreateContentPlanningFlow(Flow[ContentPlanningState]):

    @start()
    def scrape_blog_post(self):
        print(f"# Fetching draft from: {self.state.blog_post_url}")

        # Initialize FireCrawl
        app = FirecrawlApp(api_key=os.getenv("FIRECRAWL_API_KEY"))
        
        # Scrape the blog post in Markdown and HTML format
        scrape_result = app.scrape(self.state.blog_post_url,
                                       formats=['markdown', 'html'])

        # Extract the title (fallback to a UUID if not found)
        try:
            title = scrape_result.metadata.title
        except Exception:
            title = str(uuid.uuid4())

        # Store the scraped content as a markdown file
        self.state.draft_path = f'assets/{title}.md'
        with open(self.state.draft_path, 'w') as f:
            f.write(scrape_result.markdown)

        return self.state
    
    @router(scrape_blog_post)
    def select_platform(self):
        if self.state.post_type == "twitter":
            return "twitter"
        elif self.state.post_type == "linkedin":
            return "linkedin"

    @listen("twitter")
    def twitter_draft(self):
        print(f"# Planning content for: {self.state.draft_path}")
    
        # Execute the Twitter Planning Crew
        result = twitter_planning_crew.kickoff(inputs={
            'draft_path': self.state.draft_path, 
            'path_to_example_threads': self.state.path_to_example_threads
        })
    
        print(f"# Planned content for {self.state.draft_path}:")
    
        # Print each tweet in the generated thread
        for i, tweet in enumerate(result.pydantic.tweets):
            print(f"Tweet {i+1}: {tweet.content}")
            print(f"Media URLs: {tweet.media_urls}")
            print("-" * 100)
    
        return result

    @listen("linkedin")
    def linkedin_draft(self):
        print(f"# Planning content for: {self.state.draft_path}")
    
        # Execute the LinkedIn Planning Crew
        result = linkedin_planning_crew.kickoff(inputs={
            'draft_path': self.state.draft_path, 
            'path_to_example_linkedin': self.state.path_to_example_linkedin
        })
    
        print(f"# Planned content for {self.state.draft_path}:")
        print(f"{result.pydantic.content}")
    
        return result

    @listen(or_(twitter_draft, linkedin_draft))
    def save_plan(self, plan):
        path = Path("output") / "draft.json"
        with path.open('w', encoding='utf-8') as f:
            json.dump(plan.pydantic.model_dump(), f, indent=2)

In [ ]:
# class CreateSomeFlow(Flow[FlowState]):
#     @listen("opt1")
#     def opt1_task(self):
#         #task code
#         ...
#         return result1
    
#     @listen("opt2")
#     def opt2_task(self):
#         #task code
#         ...
#         return result2
    
#     @listen(or_(opt1_task, opt2_task))
#     def save_output(self, plan)
#         #some code which uses 'plan'


In [28]:
flow = CreateContentPlanningFlow()

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: CreateContentPlanningFlow                                                                                │
│  ID: d1607968-fd4a-4c67-8e20-237f62bdd29c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [29]:
print(flow.state.blog_post_url)
print(flow.state.draft_path)
print(flow.state.path_to_example_linkedin)
print(flow.state.path_to_example_threads)
print(flow.state.post_type)

from pathlib import Path

# 1) See where your notebook is running
base_dir = Path.cwd()
print(f"CWD: {base_dir}\n")

# 2) Build the full path safely
file_path = base_dir / "output" / "test.txt"  

# 3) Ensure the folder exists
file_path.parent.mkdir(parents=True, exist_ok=True)
base_dir.parent.mkdir(parents=True, exist_ok=True)

file_path.write_text("Hello from my notebook!\n", encoding="utf-8")

https://blog.dailydoseofds.com/p/5-chunking-strategies-for-rag
assets/
assets/example_linkedin.txt
assets/example_threads.txt
twitter
CWD: /Users/mukun/MyNotebooks/DDDS-Agents-4



24

In [22]:
flow.plot()

Plot saved as crewai_flow.html


In [30]:
# flow.kickoff()
await flow.kickoff_async()


#LinkedIn
# flow.state.post_type = "linkedin"
# flow.kickoff

 Flow started with ID: d1607968-fd4a-4c67-8e20-237f62bdd29c

Output()

# Fetching draft from: https://blog.dailydoseofds.com/p/5-chunking-strategies-for-rag

# Planning content for: assets/5 Chunking Strategies For RAG - by Avi Chawla.md

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: db46b4f0-567f-4b3e-9f21-e42b4c6bbe71                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f61bf7e6-65e2-4e59-a6a2-07a98460e51f                                                                     │
│  Agent: Twitter Thread Planner                                                                                  │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: db46b4f0-567f-4b3e-9f21-e42b4c6bbe71                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "topic": "5 Chunking Strategies For RAG",                                                                    │
│    "tweets": [                                                                                                  │
│      {                                                                                                          │
│        "content": "5 Chunking Strategies For RAG",                                                              │
│        "is_hook": true,                                                                                         │
│        "media_urls": []                                                                                         │
│      },                                                                                                         │
│      {                                                                                                          │
│        "content": "Why chunking is essential in RAG? 📚 It helps manage large documents by splitting them into  │
│  pieces suited for embedding model inputs. #AI #DataScience",                                                   │
│        "is_hook": false,                                                                                        │
│        "media_urls":                                                                                            │
│  ["https://substackcdn.com/image/fetch/$s_!DpHM!,w_1456,c_limit,f_auto,q_auto:good,fl_lossy/https%3A%2F%2Fsubs  │
│  tack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F6878b8fa-5e74-45a1-9a89-5aab92889126_2366x990.gif"]       │
│      },                                                                                                         │
│      {                                                                                                          │
│        "content": "1️⃣ Fixed-size Chunking: Straightforward splitting into parts by characters or words, yet it   │
│  risks breaking semantic flow. Simplicity at a cost. #MachineLearning",                                         │
│        "is_hook": false,                                                                                        │
│        "media_urls":                                                                                            │
│  ["https://substackcdn.com/image/fetch/$s_!RG5y!,w_1456,c_limit,f_auto,q_auto:good,fl_progressive:steep/https%  │
│  3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F98c422a0-f0e2-457c-a256-4476a56a601f_943x232  │
│  .png"]                                                                                                         │
│      },                                                                                                         │
│      {                                                                                                          │
│        "content": "2️⃣ Semantic Chunking: Uses embedding and cosine similarity for coherent sections. Preserves   │
│  meaning, but needs computational power. #NLP",                                                                 │
│        "is_hook": false,                           

Output()

# Planned content for assets/5 Chunking Strategies For RAG - by Avi Chawla.md:
Tweet 1: 5 Chunking Strategies For RAG
Media URLs: []
----------------------------------------------------------------------------------------------------
Tweet 2: Why chunking is essential in RAG? 📚 It helps manage large documents by splitting them into pieces suited for embedding model inputs. #AI #DataScience
Media URLs: ['https://substackcdn.com/image/fetch/$s_!DpHM!,w_1456,c_limit,f_auto,q_auto:good,fl_lossy/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F6878b8fa-5e74-45a1-9a89-5aab92889126_2366x990.gif']
----------------------------------------------------------------------------------------------------
Tweet 3: 1️⃣ Fixed-size Chunking: Straightforward splitting into parts by characters or words, yet it risks breaking semantic flow. Simplicity at a cost. #MachineLearning
Media URLs: ['https://substackcdn.com/image/fetch/$s_!RG5y!,w_1456,c_limit,f_auto,q_auto:good,fl_progressive:

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: CreateContentPlanningFlow                                                                                │
│  ID: d1607968-fd4a-4c67-8e20-237f62bdd29c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯